In [ ]:
!pip install -r requirements.txt

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access your Google API Key
# Make sure your .env file has GOOGLE_API_KEY=YOUR_API_KEY
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("Google API Key not found. Please set it in your .env file.")

# Set the environment variable for LangChain and google-generativeai
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

print("API Key loaded successfully!")

API Key loaded successfully!


In [3]:
# LangChain Imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# Other useful imports
import json # For handling structured data like our question bank
from typing import List, Dict, Any # For type hinting, improves code readability and maintainability

In [4]:
# 2.1 Initialize the Google Gemini LLM 

# Initialize the ChatGoogleGenerativeAI model
# Using 'models/gemini-1.5-flash-latest' as per  preference.
llm = ChatGoogleGenerativeAI(model="models/gemini-2.0-flash-lite", temperature=0.2) # Changed model name

print("Gemini LLM initialized successfully!")

Gemini LLM initialized successfully!


In [5]:
# 2.2 Set up LangChain Memory for Conversation

# Initialize ConversationBufferMemory to store the conversation history
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

print("LangChain Conversation Memory initialized!")

LangChain Conversation Memory initialized!


C:\Users\Parth\AppData\Local\Temp\ipykernel_3020\2795900466.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="history", return_messages=True)


In [ ]:
# 3. Excel Question Bank Definition (Loaded from JSON)

import json

def load_questions_from_json(file_path: str) -> List[Dict[str, Any]]:
    """Loads Excel interview questions from a JSON file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            questions = json.load(f)
        print(f"Loaded {len(questions)} Excel questions from {file_path}")
        return questions
    except FileNotFoundError:
        print(f"Error: Question bank file '{file_path}' not found. Please create it.")
        return []
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from '{file_path}'. Check file format.")
        return []

# Define the path to your JSON file
QUESTIONS_FILE_PATH = 'excel_questions.json'

# Load the questions
excel_questions = load_questions_from_json(QUESTIONS_FILE_PATH)

#optionally print one question to verify the structure
#print(json.dumps(excel_questions[0], indent=2))

Loaded 4 Excel questions from excel_questions.json


In [8]:
# --- STEP 4.1: ExcelEvaluator Class ---

import json
import re
from typing import Dict, Any, List

from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder # Corrected imports
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI


class ExcelEvaluator:
    def __init__(self, llm: ChatGoogleGenerativeAI):
        self.llm = llm
        # Define the prompt template for evaluating an Excel answer
        self.evaluation_template = PromptTemplate(
            input_variables=[
                "excel_question",
                "expected_answer_description",
                "evaluation_criteria",
                "candidate_answer"
            ],
            template="""You are an expert Excel interviewer and a highly precise evaluator.
Your task is to critically assess a candidate's answer to an Excel question.

Here's the Excel question asked:
"{excel_question}"

Here is the detailed expected correct answer/description from an expert:
"{expected_answer_description}"

Here are the specific criteria you must use for evaluation:
"{evaluation_criteria}"

Here is the candidate's answer:
"{candidate_answer}"

Based on the above, provide a score from 1 to 5 (where 1 is 'Poor', 3 is 'Adequate', and 5 is 'Excellent').
Then, provide a brief, objective justification for your score, highlighting strengths and weaknesses compared to the expected answer and criteria.

Format your response STRICTLY as a JSON object, without any additional text or markdown formatting (like ```json) outside of the JSON itself.
The JSON object MUST have the following two keys: "score" (integer from 1-5) and "justification" (string).
Example good output: {{"score": 4, "justification": "Good answer."}}
"""
        )

    def evaluate_answer(
        self,
        excel_question: str,
        expected_answer_description: str,
        evaluation_criteria: str,
        candidate_answer: str
    ) -> Dict[str, Any]:
        """
        Evaluates a candidate's answer to an Excel question using the LLM.

        Args:
            excel_question (str): The original Excel question.
            expected_answer_description (str): A detailed description of the correct answer.
            evaluation_criteria (str): Specific criteria for the LLM to use during evaluation.
            candidate_answer (str): The candidate's response.

        Returns:
            Dict[str, Any]: A dictionary containing the 'score' (int) and 'justification' (str).
                            Returns an error structure if parsing fails.
        """
        response_text = ""
        json_string_to_parse = ""

        try:
            # Create a runnable sequence: evaluation_template -> llm
            evaluation_chain = self.evaluation_template | self.llm

            # Generate the evaluation using the LLM
            response_text = evaluation_chain.invoke(
                {
                    "excel_question": excel_question,
                    "expected_answer_description": expected_answer_description,
                    "evaluation_criteria": evaluation_criteria,
                    "candidate_answer": candidate_answer
                }
            )

            # If the LLM returns an AIMessage, extract the content
            if hasattr(response_text, 'content'):
                response_text = response_text.content

            # Robust JSON extraction
            match = re.search(r"```json\n(.*?)```", response_text, re.DOTALL)
            if match:
                json_string_to_parse = match.group(1).strip()
            else:
                json_string_to_parse = response_text.strip()

            evaluation_result = json.loads(json_string_to_parse)

            # Basic validation of the parsed result
            if "score" not in evaluation_result or "justification" not in evaluation_result:
                raise ValueError("LLM response missing 'score' or 'justification' key.")
            if not isinstance(evaluation_result["score"], int) or not (1 <= evaluation_result["score"] <= 5):
                 raise ValueError("LLM score is not an integer between 1 and 5.")

            return evaluation_result

        except json.JSONDecodeError as e:
            print(f"--- JSON Parsing Error ---")
            print(f"Error: Could not decode JSON from LLM response. Details: {e}")
            print(f"Attempted to parse: `{json_string_to_parse}`")
            # Corrected f-string syntax here
            print(f"Original LLM Response: ```{response_text}```")
            print(f"--------------------------")
            return {"score": 0, "justification": f"Evaluation error: LLM response not valid JSON. {e}"}
        except ValueError as e:
            print(f"--- Validation Error ---")
            print(f"Error: LLM response parsing/validation failed. Details: {e}")
            print(f"Attempted to parse: `{json_string_to_parse}`")
            print(f"Original LLM Response: ```{response_text}```")
            print(f"--------------------------")
            return {"score": 0, "justification": f"Evaluation error: LLM response format invalid. {e}"}
        except Exception as e:
            print(f"--- Unexpected Error ---")
            print(f"An unexpected error occurred during evaluation: {e}")
            print(f"Original LLM Response (if available): ```{response_text}```")
            print(f"--------------------------")
            return {"score": 0, "justification": f"Unexpected evaluation error: {e}"}


print("ExcelEvaluator class defined.")

# --- Testing the ExcelEvaluator (Ensure llm and excel_questions are available) ---
if 'llm' not in locals() or 'excel_questions' not in locals() or not excel_questions:
    print("\nWARNING: 'llm' or 'excel_questions' not found. Please run Step 1, 2, and 3 cells first.")
    print("Skipping ExcelEvaluator tests.")
else:
    evaluator = ExcelEvaluator(llm)
    test_question = excel_questions[0] # Use the first question from your loaded bank

    test_candidate_answer_good = """
A relative cell reference, like `A1`, changes both its row and column parts when the formula is copied or dragged to other cells. For instance, if `=A1+B1` is in `C1` and copied to `C2`, it automatically becomes `=A2+B2`. This is incredibly useful when you want to apply the same calculation logic across a range of data, such as summing values in a column or row.

An absolute cell reference, denoted by dollar signs (e.g., `$A$1`), ensures that both the row and column parts of the cell reference remain fixed, or 'locked', when the formula is copied. This is vital when you need a formula to consistently refer back to a single, specific cell, like a fixed exchange rate, a tax percentage, or a lookup value from a master table, regardless of where the formula is placed.

Furthermore, there are **partial absolute references**:
* `A$1` (mixed reference): The column (A) changes when copied across columns, but the row (1) remains fixed when copied down rows. Useful for applying a formula across a row where a fixed header row is involved.
* `$A1` (mixed reference): The column (A) remains fixed when copied across columns, but the row (1) changes when copied down rows. Useful for applying a formula down a column where a fixed lookup column is involved.

These partial references provide immense flexibility for complex calculations across grids of data.
"""
    test_candidate_answer_poor = "Relative is A1, absolute is $A$1. Don't know when to use them."
    test_candidate_answer_average = "Relative is like A1, it moves when dragged. Absolute is $A$1, it stays put. Use relative for series, absolute for fixed lookups like a discount percentage."


    print("\n" + "="*50)
    print("--- Testing Good Answer ---")
    print("="*50)
    good_evaluation = evaluator.evaluate_answer(
        excel_question=test_question["question"],
        expected_answer_description=test_question["expected_answer_description"],
        evaluation_criteria=test_question["evaluation_criteria"],
        candidate_answer=test_candidate_answer_good
    )

    if good_evaluation and good_evaluation["score"] != 0:
        print("\n### Evaluation Result for Good Answer:")
        print(f"**Score:** {good_evaluation['score']} / 5")
        print(f"**Justification:** {good_evaluation['justification']}\n")
    else:
        print(f"\nEvaluation failed for Good Answer. Details: {good_evaluation['justification']}\n")


    print("\n" + "="*50)
    print("--- Testing Poor Answer ---")
    print("="*50)
    poor_evaluation = evaluator.evaluate_answer(
        excel_question=test_question["question"],
        expected_answer_description=test_question["expected_answer_description"],
        evaluation_criteria=test_question["evaluation_criteria"],
        candidate_answer=test_candidate_answer_poor
    )

    if poor_evaluation and poor_evaluation["score"] != 0:
        print("\n### Evaluation Result for Poor Answer:")
        print(f"**Score:** {poor_evaluation['score']} / 5")
        print(f"**Justification:** {poor_evaluation['justification']}\n")
    else:
        print(f"\nEvaluation failed for Poor Answer. Details: {poor_evaluation['justification']}\n")


    print("\n" + "="*50)
    print("--- Testing Average Answer ---")
    print("="*50)
    average_evaluation = evaluator.evaluate_answer(
        excel_question=test_question["question"],
        expected_answer_description=test_question["expected_answer_description"],
        evaluation_criteria=test_question["evaluation_criteria"],
        candidate_answer=test_candidate_answer_average
    )

    if average_evaluation and average_evaluation["score"] != 0:
        print("\n### Evaluation Result for Average Answer:")
        print(f"**Score:** {average_evaluation['score']} / 5")
        print(f"**Justification:** {average_evaluation['justification']}\n")
    else:
        print(f"\nEvaluation failed for Average Answer. Details: {average_evaluation['justification']}\n")

ExcelEvaluator class defined.

--- Testing Good Answer ---

### Evaluation Result for Good Answer:
**Score:** 5 / 5
**Justification:** The candidate demonstrates a clear and accurate understanding of both relative and absolute cell references, including their behavior when copied. The explanation is precise and easy to understand. The examples provided are appropriate and relevant to real-world scenarios. Furthermore, the candidate goes above and beyond by explaining mixed references (A$1 and $A1), showcasing a deeper understanding of Excel's capabilities. This comprehensive response exceeds the expectations of the question.


--- Testing Poor Answer ---

### Evaluation Result for Poor Answer:
**Score:** 1 / 5
**Justification:** The candidate correctly identifies the syntax for relative and absolute cell references. However, the answer demonstrates a complete lack of understanding of their behavior when copied and provides no examples of when to use each. This falls far short of the ex

In [10]:
# --- STEP 4.2: FeedbackGenerator Class ---

import json
from typing import List, Dict, Any
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI 


class FeedbackGenerator:
    def __init__(self, llm: ChatGoogleGenerativeAI):
        self.llm = llm
        self.feedback_template = PromptTemplate(
            input_variables=["interview_results"],
            template="""You are an HR professional and an Excel expert.
You have the evaluation results from an Excel mock interview.
Your task is to generate a comprehensive, constructive, and professional feedback report for the candidate.

Here are the interview results in JSON format. Each entry is for a question:
{interview_results}

The report should be clearly formatted using **Markdown** with headings and bullet points.
Ensure the tone is encouraging yet clear about areas needing development.

It must include the following sections in order:
1.  **## Overall Performance Summary**
    * Provide a qualitative assessment of their general Excel proficiency based on the scores across all questions.
2.  **## Strengths**
    * List specific Excel topics or questions where the candidate performed well (e.g., score 4 or 5), using bullet points. Summarize why they did well based on the justifications.
3.  **## Areas for Improvement**
    * List specific Excel topics or question types where the candidate struggled (e.g., score 1, 2, or 3), using bullet points.
    * For each area of improvement, provide 2-3 **actionable suggestions** for learning or practice (e.g., "Practice more with array formulas," "Review VLOOKUP's lookup limitations").
4.  **## Topic-wise Breakdown**
    * Provide a concise summary of performance for each unique Excel topic encountered (e.g., "**Formulas & Functions:** Good grasp, but needs more precision on edge cases.", "**Pivot Tables:** Found basic creation challenging."). Use bullet points for each topic.
5.  **## Overall Score & Proficiency Rating**
    * Calculate the average score across all questions and present it (e.g., "Average Score: X.X / 5").
    * Assign a general proficiency rating based on the overall performance: "Beginner", "Developing", "Intermediate", "Proficient", or "Advanced".
        * Average Score 1.0-1.9: Beginner
        * Average Score 2.0-2.9: Developing
        * Average Score 3.0-3.9: Intermediate
        * Average Score 4.0-4.4: Proficient
        * Average Score 4.5-5.0: Advanced
"""
        )

    def generate_feedback_report(self, interview_results: List[Dict[str, Any]]) -> str:
        """
        Generates a comprehensive feedback report based on interview results.

        Args:
            interview_results (List[Dict[str, Any]]): A list of dictionaries,
                                                      each containing details about a question,
                                                      the candidate's answer, and its evaluation.

        Returns:
            str: The formatted feedback report.
        """
        try:
            # Convert interview results to a pretty-printed JSON string for the LLM
            # This provides the LLM with all the necessary structured data.
            results_json_str = json.dumps(interview_results, indent=2)

            # Create an LLMChain to generate the feedback report
            # LangChainDeprecationWarning: The class `LLMChain` was deprecated... (ignored for now)
            feedback_chain = LLMChain(llm=self.llm, prompt=self.feedback_template)

            # Generate the feedback report
            # LangChainDeprecationWarning: The method `Chain.run` was deprecated... (ignored for now)
            report = feedback_chain.run(interview_results=results_json_str)
            return report
        except Exception as e:
            print(f"An error occurred during feedback generation: {e}")
            return "Failed to generate feedback report due to an internal error."

print("FeedbackGenerator class defined.")


# --- Testing the FeedbackGenerator ---
'''if 'llm' not in locals():
    print("\nWARNING: 'llm' not found. Please run Step 2.1 cell first.")
    print("Skipping FeedbackGenerator tests.")
else:
    feedback_gen = FeedbackGenerator(llm)

    # Create SAMPLE interview results to test the FeedbackGenerator
    # This simulates the output we'd get from the ExcelEvaluator for multiple questions
    sample_interview_results = [
        {
            "id": "EXL001",
            "topic": "Formulas & Functions",
            "question": "Explain relative vs absolute cell references...",
            "candidate_answer": "Relative moves, absolute stays.",
            "evaluation": {"score": 5, "justification": "Candidate provided a perfectly comprehensive explanation, including partial absolute references and clear use cases. Demonstrated excellent understanding."}
        },
        {
            "id": "EXL002",
            "topic": "Data Analysis & Pivot Tables",
            "question": "Describe steps for PivotTable and purpose of Values area.",
            "candidate_answer": "Click insert, pivot table. Values is for sums.",
            "evaluation": {"score": 3, "justification": "Candidate understood the basic steps for PivotTable creation and the high-level purpose of the Values area. However, the explanation lacked detail on specific steps and various aggregation functions beyond sums, indicating an adequate but not comprehensive understanding."}
        },
        {
            "id": "EXL003",
            "topic": "Conditional Formatting",
            "question": "How to highlight sales above average?",
            "candidate_answer": "Select data, home tab, conditional formatting, above average rule.",
            "evaluation": {"score": 4, "justification": "Candidate provided accurate and concise steps for applying 'Above Average' conditional formatting. Demonstrated practical knowledge. A minor detail missing was mention of selecting the formatting style, but otherwise solid."}
        },
        {
            "id": "EXL004",
            "topic": "VLOOKUP/XLOOKUP",
            "question": "Explain VLOOKUP scenario, limitations, and XLOOKUP alternative.",
            "candidate_answer": "VLOOKUP looks left to right. XLOOKUP is better.",
            "evaluation": {"score": 2, "justification": "Candidate identified one key limitation of VLOOKUP (left-to-right) and correctly named XLOOKUP as an alternative. However, the explanation of a VLOOKUP scenario was absent, and the description of XLOOKUP's advantages was extremely brief, missing most crucial improvements. Demonstrates weak understanding."}
        }
    ]

    print("\n" + "="*50)
    print("--- Generating Sample Feedback Report ---")
    print("="*50)
    sample_report = feedback_gen.generate_feedback_report(sample_interview_results)
    print(sample_report)
    print("="*50 + "\n")'''

FeedbackGenerator class defined.


'if \'llm\' not in locals():\n    print("\nWARNING: \'llm\' not found. Please run Step 2.1 cell first.")\n    print("Skipping FeedbackGenerator tests.")\nelse:\n    feedback_gen = FeedbackGenerator(llm)\n\n    # Create SAMPLE interview results to test the FeedbackGenerator\n    # This simulates the output we\'d get from the ExcelEvaluator for multiple questions\n    sample_interview_results = [\n        {\n            "id": "EXL001",\n            "topic": "Formulas & Functions",\n            "question": "Explain relative vs absolute cell references...",\n            "candidate_answer": "Relative moves, absolute stays.",\n            "evaluation": {"score": 5, "justification": "Candidate provided a perfectly comprehensive explanation, including partial absolute references and clear use cases. Demonstrated excellent understanding."}\n        },\n        {\n            "id": "EXL002",\n            "topic": "Data Analysis & Pivot Tables",\n            "question": "Describe steps for PivotT

In [11]:
# --- STEP 5: Interview Logic (MockInterviewer Class) - FINAL DEFINITIVE FIX FOR CONVERSATIONAL FLOW ---

from typing import List, Dict, Any
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.memory import ConversationBufferMemory 
from langchain.schema import HumanMessage, AIMessage, SystemMessage 
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import datetime


class MockInterviewer:
    def __init__(self, llm: ChatGoogleGenerativeAI, memory: ConversationBufferMemory, questions: List[Dict[str, Any]]):
        self.llm = llm
        self.memory = memory 
        self.questions = questions
        self.evaluator = ExcelEvaluator(llm)
        self.feedback_generator = FeedbackGenerator(llm)
        self.interview_history: List[Dict[str, Any]] = [] # To store Q&A with evaluations
        self.current_question_index = 0

        # --- File Logging Setup ---
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.transcript_dir = "interview_transcripts"
        self.feedback_dir = "interview_feedback"
        os.makedirs(self.transcript_dir, exist_ok=True)
        os.makedirs(self.feedback_dir, exist_ok=True)

        self.transcript_filename = os.path.join(self.transcript_dir, f"transcript_{timestamp}.txt")
        self.feedback_filename = os.path.join(self.feedback_dir, f"feedback_{timestamp}.txt")

        self.transcript_file = open(self.transcript_filename, "w", encoding="utf-8")
        # --- End File Logging Setup ---

        # Define a general system persona for the AI's conversational turns
        self.system_persona = SystemMessage(
            content="You are an AI-powered Excel interviewer. Your role is to conduct a structured mock interview to assess a candidate's Excel skills. Be professional, clear, and guide the candidate through the process. Do not answer Excel questions yourself, only ask them and acknowledge answers based on the provided instructions."
        )

    def _display_message(self, speaker: str, message: str):
        """Helper to print messages to console and log to file."""
        display_text = f"\n\033[1;34mAI Interviewer:\033[0m {message}" if speaker == "AI" else f"\n\033[1;32mCandidate:\033[0m {message}"
        log_text = f"\n{speaker}: {message}" # Plain text for log file

        print(display_text)
        if self.transcript_file:
            self.transcript_file.write(log_text + "\n")


    def _get_ai_response(self, user_input_text: str) -> str:
        """Gets a conversational response from the LLM based on user input and history."""
        # Manually construct messages including system persona and memory content
        messages = [self.system_persona] + self.memory.buffer_as_messages + [HumanMessage(content=user_input_text)]
        
        # Invoke the LLM directly
        ai_response = self.llm.invoke(messages).content
        
        # Add the human and AI messages to memory for the next turn
        self.memory.chat_memory.add_user_message(user_input_text)
        self.memory.chat_memory.add_ai_message(ai_response)
        
        return ai_response

    def start_interview(self):
        """Introduces the interviewer and explains the process."""
        self._display_message("AI", "Hello! I'm your AI-powered Excel Mock Interviewer.")
        
        # LLM provides a single, coherent introduction
        intro_prompt = f"Introduce yourself as an AI Excel interviewer. Explain that you will ask {len(self.questions)} questions one by one to assess Excel proficiency. Mention that responses will be evaluated and a comprehensive feedback report will be provided at the end. Do NOT ask if the candidate is ready yet, just provide this full explanation."
        
        full_introduction = self._get_ai_response(intro_prompt)
        self._display_message("AI", full_introduction)


    def ask_excel_question(self, question_data: Dict[str, Any], question_num: int, total_questions: int):
        """Asks a specific Excel question to the candidate."""
        self._display_message("AI", f"--- Question {question_num} of {total_questions}: {question_data['topic']} ---")
        
        # LLM generates the question phrasing based on the actual question text
        question_phrasing_prompt = f"Please phrase the following Excel question clearly and professionally for a candidate: {question_data['question']}"
        llm_question_output = self.llm.invoke([self.system_persona, HumanMessage(content=question_phrasing_prompt)]).content # No memory needed for this specific phrasing
        
        self._display_message("AI", llm_question_output)


    def acknowledge_and_process_answer(self, candidate_answer: str):
        """Acknowledges the candidate's answer and prepares for next step."""
        # LLM acknowledges the candidate's *actual* answer
        acknowledgement_message_prompt = f"The candidate has just provided their answer: '{candidate_answer[:100]}...'. Please acknowledge their response briefly and professionally, without evaluating it, and indicate that you are processing it before the next question or concluding."
        acknowledgement_message = self._get_ai_response(acknowledgement_message_prompt)
        self._display_message("AI", acknowledgement_message)


    def end_interview(self):
        """Concludes the interview, generates feedback, and saves logs."""
        self._display_message("AI", "Thank you for completing the mock interview!")
        
        # LLM generates final graceful conclusion
        final_message_prompt = "Gracefully conclude the interview. Inform the candidate that you have evaluated all their responses and are now compiling their comprehensive feedback report. Express appreciation for their time."
        final_message = self._get_ai_response(final_message_prompt)
        self._display_message("AI", final_message)

        self._display_message("AI", "--- Generating Your Feedback Report ---")
        report = self.feedback_generator.generate_feedback_report(self.interview_history)

        self._display_message("AI", "--- Your Performance Feedback ---")
        print(report) # Still print to console for immediate viewing

        # --- Save Feedback to File ---
        try:
            with open(self.feedback_filename, "w", encoding="utf-8") as f:
                f.write(report)
            self._display_message("AI", f"Feedback report saved to: {self.feedback_filename}")
        except Exception as e:
            self._display_message("AI", f"Error saving feedback report: {e}")
        # --- End Save Feedback ---

        self._display_message("AI", "--- End of Interview ---")


    def run_interview(self):
        """Orchestrates the entire interview flow."""
        try:
            self.start_interview()

            self._display_message("AI", "Are you ready to start the questions now? (yes/no)")
            ready = input("Your input: ").strip().lower()
            self._display_message("Candidate", ready) # This logs the 'yes/no' response

            if ready != 'yes':
                self._display_message("AI", "No problem. We can proceed later. Goodbye!")
                return

            # --- NEW CODE: Prompt for Name ---
            self._display_message("AI", "Great! What is your name?")
            candidate_name_input = input("Your Name: ").strip()
            if candidate_name_input:
                self.candidate_name = candidate_name_input
                self._display_message("Candidate", self.candidate_name) # Log the name
                self._display_message("AI", f"Nice to meet you, {self.candidate_name}! Let's begin.")
            else:
                self.candidate_name = "Candidate" # Default if no name entered
                self._display_message("AI", "No problem. We'll proceed. Let's begin.")
            # --- END NEW CODE ---

            while self.current_question_index < len(self.questions):
                current_question_data = self.questions[self.current_question_index]
                question_num = self.current_question_index + 1
                total_questions = len(self.questions)

                self.ask_excel_question(current_question_data, question_num, total_questions)

                candidate_answer = input("Your Answer: ").strip()
                self._display_message("Candidate", candidate_answer)

                self.acknowledge_and_process_answer(candidate_answer)

                if not candidate_answer:
                    self._display_message("AI", "It seems you didn't provide an answer. Moving to evaluation for this question.")
                    candidate_answer = "[No response provided]"

                evaluation_result = self.evaluator.evaluate_answer(
                    excel_question=current_question_data["question"],
                    expected_answer_description=current_question_data["expected_answer_description"],
                    evaluation_criteria=current_question_data["evaluation_criteria"],
                    candidate_answer=candidate_answer
                )

                self.interview_history.append({
                    "id": current_question_data["id"],
                    "topic": current_question_data["topic"],
                    "question": current_question_data["question"],
                    "candidate_answer": candidate_answer,
                    "evaluation": evaluation_result
                })
                self.current_question_index += 1

            self.end_interview()
            self.memory.clear()

        finally:
            self._display_message("AI", f"Transcript saved to: {self.transcript_filename}")
            if self.transcript_file and not self.transcript_file.closed:
                self.transcript_file.close()

print("MockInterviewer class defined.")

# --- Running the Mock Interview ---
'''if 'llm' not in locals() or 'memory' not in locals() or 'excel_questions' not in locals() or not excel_questions:
    print("\nWARNING: Missing dependencies. Please run Step 1, 2, and 3 cells first.")
    print("Cannot run Mock Interview.")
else:
    print("\n" + "="*70)
    print("          STARTING AI EXCEL MOCK INTERVIEW SIMULATION")
    print("="*70)

    interviewer = MockInterviewer(llm, memory, excel_questions)
    interviewer.run_interview()

    print("\n" + "="*70)
    print("          AI EXCEL MOCK INTERVIEW SIMULATION FINISHED")
    print("="*70)'''

MockInterviewer class defined.


'if \'llm\' not in locals() or \'memory\' not in locals() or \'excel_questions\' not in locals() or not excel_questions:\n    print("\nWARNING: Missing dependencies. Please run Step 1, 2, and 3 cells first.")\n    print("Cannot run Mock Interview.")\nelse:\n    print("\n" + "="*70)\n    print("          STARTING AI EXCEL MOCK INTERVIEW SIMULATION")\n    print("="*70)\n\n    interviewer = MockInterviewer(llm, memory, excel_questions)\n    interviewer.run_interview()\n\n    print("\n" + "="*70)\n    print("          AI EXCEL MOCK INTERVIEW SIMULATION FINISHED")\n    print("="*70)'